In [1]:
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import Dropdown, HTML,HTMLMath, interact, IntSlider, FloatSlider,widgets, Image, HBox,VBox,Tab,Label,Text,Textarea,Layout,Accordion, Button
from IPython.display import display,clear_output, update_display, Javascript
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.patches as mpatches
from math import ceil

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# <center style='color:red;'><b> L'inférence Bayésienne </b></center>

In [ ]:
A1 = Accordion()
children1=[]

ls = Layout(width='95%', height='25px')

VP = IntSlider(
    value = 99,
    min = 1,
    max = 1000,
    step = 1,
    style = {"handle_color":'#D00000'},
    description = "vrai positif",
    readout=False,
    layout = ls)
VN = IntSlider(
    value = 196,
    min = 1,
    max = 1000,
    step = 1,
    style = {"handle_color":'#B0E0B0'},
    description = "vrai negatif",
    readout=False,
    layout = ls)
FP = IntSlider(
    value = 4,
    min = 1,
    max = 1000,
    step = 1,
    style = {"handle_color":'#00D000'},
    description = "faux positif",
    readout=False,
    layout = ls)
FN = IntSlider(
    value = 1,
    min = 1,
    max = 1000,
    step = 1,
    style = {"handle_color":'#E0B0B0'},
    description = "faux negatif",
    readout=False,
    layout = ls)
l = Layout(width='120px', height='25px')
lp = Layout(width='25px',height='25px')
ld = Layout(width='120px', height='50px')
ll = Layout(width='220px', height='25px')
l2 = Layout(width='160px', height='25px')



#print(VP.style.keys)

colonne1 = [HTML("<center><b></b></center>",layout=l),
           HTML("<center><b>Test Positif</b></center>",layout=l),
           HTML("<center><b>Test Negatif</b></center>",layout=l)]
colonne2 = [HTML("<center><b>Personne Malade</b></center>",layout=l),
           HTML("<center style='background-color:#D00000'>99</center>",layout=l),
           HTML("<center style='background-color:#E0B0B0'>1</center>",layout=l)]
colonne3 = [HTML("<center><b>Personne Saine</b></center>",layout=l),
           HTML("<center style='background-color:#00D000'>4</center>",layout=l),
           HTML("<center style='background-color:#B0E0B0'>196</center>",layout=l)]
tableau = [VBox(colonne1), VBox(colonne2), VBox(colonne3)]

M = [HTML("M = Malades ="),colonne2[1],
           HTML("<center>+</center>", layout=lp),colonne2[2],HTML("= 100")]
S = [HTML("S = Sains ="),colonne3[1],
           HTML("<center>+</center>", layout=lp),colonne3[2],HTML("= 200")]

P = [HTML("P = Tests positifs ="),colonne2[1],
           HTML("<center>+</center>", layout=lp),colonne3[1],HTML("= 103")]
N = [HTML("N = Tests négatifs ="),colonne2[2],
           HTML("<center>+</center>", layout=lp),colonne3[2],HTML("= 197")]

Se = [HTML("Se = sensibilité =", layout=l),VBox([colonne2[1],HTML("<svg height='10' width='300'><line x1='0' y1='0' x2='300' y2='0' style='stroke:rgb(0,0,0);stroke-width:5.2' /></svg>", layout=Layout(width='300px',height='10px')),
           HBox([colonne2[1],HTML("<center>+</center>", layout=lp),colonne2[2]],layout=Layout(width='280px',height='35px',display='flex',
                    flex_flow='row',
                    align_items='center',
                    align_content = 'center'))],layout=Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    align_content = 'center')),HTML("= 99.00 %",layout=l)]
Sp = [HTML("Sp = spécificité =", layout=l),VBox([colonne3[2],HTML("<svg height='10' width='300'><line x1='0' y1='0' x2='300' y2='0' style='stroke:rgb(0,0,0);stroke-width:5.2' /></svg>", layout=Layout(width='300px',height='10px')),
           HBox([colonne3[1],HTML("<center>+</center>", layout=lp),colonne3[2]],layout=Layout(width='280px',height='35px',display='flex',
                    flex_flow='row',
                    align_items='center',
                    align_content = 'center'))],layout=Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    align_content = 'center')),HTML("= 98.00 %",layout=l)]

Pr = [HTML("Pr = prévalence =", layout=l),
      VBox([HBox([colonne2[1],
                  HTML("<center>+</center>", layout=lp),
                  colonne2[2]],
                  layout=Layout(width='280px',height='35px',display='flex',
                                            flex_flow='row',
                                            align_items='center',
                                            align_content = 'center')),
            HTML("<svg height='10' width='600'><line x1='0' y1='0' x2='600' y2='0' style='stroke:rgb(0,0,0);stroke-width:5.2' /></svg>", layout=Layout(width='600px',height='10px')),
            HBox([colonne2[1],
                  HTML("<center>+</center>", layout=lp),
                  colonne2[2],
                  HTML("<center>+</center>", layout=lp),
                  colonne3[1],
                  HTML("<center>+</center>", layout=lp),
                  colonne3[2]],
                  layout=Layout(width='580px',height='35px',display='flex',
                                            flex_flow='row',
                                            align_items='center',
                                            align_content = 'center'))],
            layout=Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    align_content = 'center')),
      HTML("= 33.33 %",layout=l)]

wimage_b = Image(
        value=b"",
        format='png',
        width='900px',
        height='450px',
        )

VPP = [HTML("VPP = valeur prédictive positive =", layout=ll),
      VBox([colonne2[1],
            HTML("<svg height='10' width='300'><line x1='0' y1='0' x2='300' y2='0' style='stroke:rgb(0,0,0);stroke-width:5.2' /></svg>", layout=Layout(width='300px',height='10px')),
            HBox([colonne2[1],
                  HTML("<center>+</center>", layout=lp),
                  colonne3[1]],
                  layout=Layout(width='280px',height='35px',display='flex',
                                            flex_flow='row',
                                            align_items='center',
                                            align_content = 'center'))],
            layout=Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    align_content = 'center')),
      HTML("= 96.12 %",layout=l)]

VPN = [HTML("VPN = valeur prédictive négative =", layout=ll),
      VBox([colonne3[2],
            HTML("<svg height='10' width='300'><line x1='0' y1='0' x2='300' y2='0' style='stroke:rgb(0,0,0);stroke-width:5.2' /></svg>", layout=Layout(width='300px',height='10px')),
            HBox([colonne2[2],
                  HTML("<center>+</center>", layout=lp),
                  colonne3[2]],
                  layout=Layout(width='280px',height='35px',display='flex',
                                            flex_flow='row',
                                            align_items='center',
                                            align_content = 'center'))],
            layout=Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    align_content = 'center')),
      HTML("= 99.49 %",layout=l)]

Dc = [HTML("Dc = diagnostique correct =", layout=l2),
      VBox([HBox([colonne2[1],
                  HTML("<center>+</center>", layout=lp),
                  colonne3[2]],
                  layout=Layout(width='280px',height='35px',display='flex',
                                            flex_flow='row',
                                            align_items='center',
                                            align_content = 'center')),
            HTML("<svg height='10' width='600'><line x1='0' y1='0' x2='600' y2='0' style='stroke:rgb(0,0,0);stroke-width:5.2' /></svg>", layout=Layout(width='600px',height='10px')),
            HBox([colonne2[1],
                  HTML("<center>+</center>", layout=lp),
                  colonne2[2],
                  HTML("<center>+</center>", layout=lp),
                  colonne3[1],
                  HTML("<center>+</center>", layout=lp),
                  colonne3[2]],
                  layout=Layout(width='580px',height='35px',display='flex',
                                            flex_flow='row',
                                            align_items='center',
                                            align_content = 'center'))],
            layout=Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    align_content = 'center')),
      HTML("= 98.33 %",layout=l)]

De = [HTML("De = diagnostique erroné =", layout=l2),
      VBox([HBox([colonne2[2],
                  HTML("<center>+</center>", layout=lp),
                  colonne3[1]],
                  layout=Layout(width='280px',height='35px',display='flex',
                                            flex_flow='row',
                                            align_items='center',
                                            align_content = 'center')),
            HTML("<svg height='10' width='600'><line x1='0' y1='0' x2='600' y2='0' style='stroke:rgb(0,0,0);stroke-width:5.2' /></svg>", layout=Layout(width='600px',height='10px')),
            HBox([colonne2[1],
                  HTML("<center>+</center>", layout=lp),
                  colonne2[2],
                  HTML("<center>+</center>", layout=lp),
                  colonne3[1],
                  HTML("<center>+</center>", layout=lp),
                  colonne3[2]],
                  layout=Layout(width='580px',height='35px',display='flex',
                                            flex_flow='row',
                                            align_items='center',
                                            align_content = 'center'))],
            layout=Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    align_content = 'center')),
      HTML("= 1.67 %",layout=l)]

def figure():
    tot = VP.value + VN.value + FP.value + FN.value
    pr = 100*(VP.value + FN.value)/tot
    sp = 100*VN.value/(VN.value+FP.value)
    se = 100*VP.value/(VP.value+FN.value)
    if pr>99:
        pr = 99
    elif pr < 1:
        pr = 1
    if sp >99:
        sp = 99
    elif sp < 1:
        sp = 1
    if se > 99:
        se = 99
    elif se < 1:
        se = 1
    rect1 = mpatches.Rectangle((pr,0),100-pr,sp,linewidth=1, ec="none",fc="#B0E0B0")
    rect2 = mpatches.Rectangle((pr,sp),100-pr,100-sp,linewidth=1, ec="none",fc="#00D000")
    rect3 = mpatches.Rectangle((0,0),pr,100-se,linewidth=1, ec="none",fc="#E0B0B0")
    rect4 = mpatches.Rectangle((0,100-se),pr,se,linewidth=1, ec="none",fc="#D00000")
    plt.plot([],[])
    plt.xlim([-10,110])
    plt.ylim([-10,110])
    plt.text(pr/2,-5,"Pr",horizontalalignment='center',verticalalignment='center')
    plt.text(50+pr/2,-5,"1-Pr",horizontalalignment='center',verticalalignment='center')
    plt.text(-5,50-se/2,"1-Se",horizontalalignment='center',verticalalignment='center')
    plt.text(-5,100-se/2,"Se",horizontalalignment='center',verticalalignment='center')
    plt.text(105,sp/2,"Sp",horizontalalignment='center',verticalalignment='center')
    plt.text(105,50+sp/2,"1-Sp",horizontalalignment='center',verticalalignment='center')
    ax = plt.gca()
    ax.spines['right'].set_color(None)
    ax.spines['left'].set_color(None)
    ax.spines['top'].set_color(None)
    ax.spines['bottom'].set_color(None)
    plt.xticks([])
    plt.yticks([])
    ax.add_patch(rect1)
    ax.add_patch(rect2)
    ax.add_patch(rect3)
    ax.add_patch(rect4)
    plt.savefig("prevalence.png")
    plt.clf()
    plt.close()
    
    file_b = open("./prevalence.png",'rb')
    image_b= file_b.read()
    wimage_b.value = image_b

figure()

def change(c):
    colonne2[1].value = "<center style='background-color:#D00000'>{}</center>".format(VP.value)
    colonne2[2].value = "<center style='background-color:#E0B0B0'>{}</center>".format(FN.value)
    colonne3[1].value = "<center style='background-color:#00D000'>{}</center>".format(FP.value)
    colonne3[2].value = "<center style='background-color:#B0E0B0'>{}</center>".format(VN.value)
    S[-1].value = "= {}".format(VN.value+FP.value)
    M[-1].value = "= {}".format(VP.value+FN.value)
    P[-1].value = "= {}".format(VP.value+FP.value)
    N[-1].value = "= {}".format(VN.value+FN.value)
    Se[-1].value = "= {:.2f} %".format(100*VP.value/(VP.value+FN.value))
    Sp[-1].value = "= {:.2f} %".format(100*VN.value/(VN.value+FP.value))
    Pr[-1].value = "= {:.2f} %".format(100*(VP.value+FN.value)/(VP.value+VN.value+FP.value+FN.value))
    VPP[-1].value = "= {:.2f} %".format(100*VP.value/(VP.value+FP.value))
    VPN[-1].value = "= {:.2f} %".format(100*VN.value/(VN.value+FN.value))
    Dc[-1].value = "= {:.2f} %".format(100*(VN.value+VP.value)/(VP.value+VN.value+FP.value+FN.value))
    De[-1].value = "= {:.2f} %".format(100*(FN.value+FP.value)/(VP.value+VN.value+FP.value+FN.value))
    figure()

VP.observe(change, names='value')
VN.observe(change, names='value')
FP.observe(change, names='value')
FN.observe(change, names='value')

A11 = Accordion()
children11 = []

children11.append(VBox([HBox(M), HTML("",layout=l),HBox(S)]))
children11.append(VBox([HBox(P), HTML("",layout=l),HBox(N)]))
children11.append(VBox([HTML("<b>La sensibilité</b> est le pourcentage de tests positifs sur une population malade"),
                        HBox(Se,layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='center',
                    align_content = 'center')),
                        HTML("",layout=ld),
                        HTML("<b>La spécificité</b> est le pourcentage de tests négatifs sur une population saine"),
                        HBox(Sp,layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='center',
                    align_content = 'center'))]))
children11.append(VBox([HTML("<b>La prévalence</b> est le pourcentage de de personnes malades dans l'échantillon de population testé"),
                        HBox(Pr,layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='center',
                    align_content = 'center')),
                       wimage_b]))
children11.append(VBox([HTML("<b>La valeur prédictive positive VPP</b> est le pourcentage de chance d'être malade lorsque le test réalisé est positif"),
                       HBox(VPP,layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='center',
                    align_content = 'center')),
                       HTML("",layout=ld),
                       HTML("<b>La valeur prédictive négative VPN</b> est le pourcentage de chance d'être sain lorsque le test réalisé est négatif"),
                       HBox(VPN,layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='center',
                    align_content = 'center'))]))
children11.append(VBox([HTML("Le pourcentage de diagnostiques corrects peut se calculer en utilisant le nombre de bons diagnostiques (VP et VN) et le nombre total de tests réalisés."),
                       HBox(Dc,layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='center',
                    align_content = 'center')),
                       HTML("",layout=ld),
                       HTML("Le pourcentage de diagnostiques erronés peut se calculer en utilisant le nombre de mauvais diagnostiques (FP et FN) et le nombre total de tests réalisés."),
                       HBox(De,layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='center',
                    align_content = 'center'))]))

A11.children = children11
A11.set_title(0,"1.1. Nombre de malades et nombre de personnes bien portantes")
A11.set_title(1,"1.2. Nombre de tests positifs et de tests négatifs")
A11.set_title(2,"1.3. Sensibilité et spécificité")
A11.set_title(3,"1.4. Prévalence")
A11.set_title(4,"1.5. Valeur prédictive positive et valeur prédictive négative")
A11.set_title(5,"1.6. Diagnostique correct/erroné")
A11.selected_index=None


Se2 = FloatSlider(
    value = 99.,
    min = 60.,
    max = 99.9,
    step = 0.1,
    style = {"handle_color":'red'},
    description = "Sélectivité",
    readout=True,
    layout = ls)

Sp2 = FloatSlider(
    value = 98.,
    min = 50.,
    max = 99.9,
    step = 0.1,
    style = {"handle_color":'green'},
    description = "Spécificité",
    readout=True,
    layout = ls)

Se2_res = HTML("<b>La selectivité</b> du test est de {:.2f} %".format(99.))
Sp2_res = HTML("<b>La spécificité</b> du test est de {:.2f} %".format(98.))

colonne4 = [HTML("<center><b></b></center>",layout=l),
           HTML("<center><b>Test Positif</b></center>",layout=l),
           HTML("<center><b>Test Negatif</b></center>",layout=l)]
colonne5 = [HTML("<center><b>Personne Malade</b></center>",layout=l),
           HTML("<center style='background-color:#D00000'>1236</center>",layout=l),
           HTML("<center style='background-color:#E0B0B0'>12</center>",layout=l)]
colonne6 = [HTML("<center><b>Personne Saine</b></center>",layout=l),
           HTML("<center style='background-color:#00D000'>79</center>",layout=l),
           HTML("<center style='background-color:#B0E0B0'>3848</center>",layout=l)]
tableau2 = [VBox(colonne4), VBox(colonne5), VBox(colonne6)]

lls = Layout(width='95%', height='25px')


P2 = IntSlider(
    value = 40287,
    min = 100,
    max = 100000,
    step = 1,
    style = {"handle_color":'#D0D000'},
    description = "Test positif",
    readout=True,
    layout = lls)
N2 = IntSlider(
    value = 87254,
    min = 100,
    max = 100000,
    step = 1,
    style = {"handle_color":'#E0E0B0'},
    description = "Test negatif",
    readout=True,
    layout = lls)


colonne7 = [HTML("<center><b></b></center>",layout=l),
           HTML("<center><b>Test Positif</b></center>",layout=l),
           HTML("<center><b>Test Negatif</b></center>",layout=l)]
colonne8 = [HTML("<center><b>Personne Malade</b></center>",layout=l),
           HTML("<center style='background-color:#D00000'>?</center>",layout=l),
           HTML("<center style='background-color:#E0B0B0'>?</center>",layout=l)]
colonne9 = [HTML("<center><b>Personne Saine</b></center>",layout=l),
           HTML("<center style='background-color:#00D000'>?</center>",layout=l),
           HTML("<center style='background-color:#B0E0B0'>?</center>",layout=l)]
colonne10 = [HTML("<center><b>Total</b></center>",layout=l),
           HTML("<center>40287</center>",layout=l),
           HTML("<center>87254</center>",layout=l)]
tableau3 = [VBox(colonne7), VBox(colonne8), VBox(colonne9),VBox(colonne10)]

colonne11 = [HTML("<center><b></b></center>",layout=l),
           HTML("<center><b>Test Positif</b></center>",layout=l),
           HTML("<center><b>Test Negatif</b></center>",layout=l),
           HTML("<center><b>Total</b></center>",layout=l)]
colonne12 = [HTML("<center><b>Personne Malade</b></center>",layout=l),
           HTML("<center style='background-color:#D00000'>?</center>",layout=l),
           HTML("<center style='background-color:#E0B0B0'>?</center>",layout=l),
           HTML("<center>38903</center>",layout=l)]
colonne13 = [HTML("<center><b>Personne Saine</b></center>",layout=l),
           HTML("<center style='background-color:#00D000'>?</center>",layout=l),
           HTML("<center style='background-color:#B0E0B0'>?</center>",layout=l),
           HTML("<center>88638</center>",layout=l)]
colonne14 = [HTML("<center><b>Total</b></center>",layout=l),
           HTML("<center>40287</center>",layout=l),
           HTML("<center>87254</center>",layout=l),
           HTML("<center></center>",layout=l)]
tableau4 = [VBox(colonne11), VBox(colonne12), VBox(colonne13),VBox(colonne14)]

colonne15 = [HTML("<center><b></b></center>",layout=l),
           HTML("<center><b>Test Positif</b></center>",layout=l),
           HTML("<center><b>Test Negatif</b></center>",layout=l)]
colonne16 = [HTML("<center><b>Personne Malade</b></center>",layout=l),
           HTML("<center style='background-color:#D00000'>38514</center>",layout=l),
           HTML("<center style='background-color:#E0B0B0'>389</center>",layout=l)]
colonne17 = [HTML("<center><b>Personne Saine</b></center>",layout=l),
           HTML("<center style='background-color:#00D000'>1773</center>",layout=l),
           HTML("<center style='background-color:#B0E0B0'>86865</center>",layout=l)]

tableau5 = [VBox(colonne15), VBox(colonne16), VBox(colonne17)]

Pr2 = [HTML("Pr = prévalence =", layout=l),
      VBox([HBox([colonne12[3]],
                  layout=Layout(width='140px',height='35px',display='flex',
                                            flex_flow='row',
                                            align_items='center',
                                            align_content = 'center')),
            HTML("<svg height='10' width='300'><line x1='0' y1='0' x2='300' y2='0' style='stroke:rgb(0,0,0);stroke-width:5.2' /></svg>", layout=Layout(width='300px',height='10px')),
            HBox([colonne12[3],
                  HTML("<center>+</center>", layout=lp),
                  colonne13[3]],
                  layout=Layout(width='280px',height='35px',display='flex',
                                            flex_flow='row',
                                            align_items='center',
                                            align_content = 'center'))],
            layout=Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    align_content = 'center')),
      HTML("= 30.50 %",layout=l)]

VPP2 = [HTMLMath(r"""On peut calculer la valeur prédictive positive VPP car on connait <b>le nombre de vrais positifs VP</b> et le nombre de <b>tests positifs P</b> dans l'échantillon:
$$VPP = \frac{VP}{P} = """+r"{:.2f}\ \% $$".format(95.6)),
        HTML("",layout=l),
        HTMLMath(r"""On peut calculer la valeur prédictive négative VPN car on connait <b>le nombre de vrais négatifs VN</b> et le nombre de <b>tests négatifs N</b> dans l'échantillon:
$$VPN = \frac{VN}{N} = """+r"{:.2f}\ \% $$".format(99.55))]

DE2 = [HTMLMath(r"""On peut calculer le pourcentage de diagnostiques corrects car on connait <b>le nombre de vrais positifs VP</b> et <b>le nombre de vrais négatifs VN</b> dans l'échantillon:
$$Dc = \frac{VP + VN}{Nombre\ total\ de\ tests} = """+r"{:.2f}\ \% $$".format(98.3)),
        HTML("",layout=l),
        HTMLMath(r"""On peut calculer le pourcentage de diagnostiques erronés car on connait <b>le nombre de faux positifs FP</b> et <b>le nombre de faux négatifs FN</b> dans l'échantillon:
$$De = \frac{FP + FN}{Nombre\ total\ de\ tests} = """+r"{:.2f}\ \% $$".format(1.7))]

def change2(c):
    colonne5[1].value = "<center style='background-color:#D00000'>{}</center>".format(int(Se2.value*1248./100.))
    colonne5[2].value = "<center style='background-color:#E0B0B0'>{}</center>".format(int((100.-Se2.value)*1248./100.))
    colonne6[1].value = "<center style='background-color:#00D000'>{}</center>".format(int((100.-Sp2.value)*3927./100.))
    colonne6[2].value = "<center style='background-color:#B0E0B0'>{}</center>".format(int(Sp2.value*3927./100.))
    Se2_res.value ="<b>La selectivité</b> du test est de {:.2f} %".format(Se2.value)
    Sp2_res.value = "<b>La spécificité</b> du test est de {:.2f} %".format(Sp2.value)
    change3()
    
def change3(c=None):
    P2.min = max(10,ceil(N2.value*(100.-Sp2.value)/Sp2.value))
    N2.min = max(10,ceil(P2.value*(100.-Se2.value)/Se2.value))
    colonne10[1].value = "<center>{}</center>".format(P2.value)
    colonne10[2].value = "<center>{}</center>".format(N2.value)
    colonne14[1].value = "<center>{}</center>".format(P2.value)
    colonne14[2].value = "<center>{}</center>".format(N2.value)
    ma = int((P2.value-(100.-Sp2.value)*N2.value/Sp2.value)/(Se2.value/100.-(1.-Se2.value/100.)*(100.-Sp2.value)/Sp2.value))
    sa = P2.value+N2.value-ma
    colonne12[3].value = "<center>{}</center>".format(ma)
    colonne13[3].value = "<center>{}</center>".format(sa)
    Pr2[-1].value = "= {:.2f} %".format(100.*ma/(sa+ma))
    colonne16[1].value = "<center style='background-color:#D00000'>{:.0f}</center>".format(Se2.value*ma/100.)
    colonne16[2].value = "<center style='background-color:#E0B0B0'>{:.0f}</center>".format((100.-Se2.value)*ma/100.)
    colonne17[1].value = "<center style='background-color:#00D000'>{:.0f}</center>".format(P2.value-Se2.value*ma/100.)
    colonne17[2].value = "<center style='background-color:#B0E0B0'>{:.0f}</center>".format(N2.value-(100.-Se2.value)*ma/100.)
    VPP2[0].value = r"""On peut calculer la valeur prédictive positive VPP car on connait <b>le nombre de vrais positifs VP</b> et le nombre de <b>tests positifs P</b> dans l'échantillon:
$$VPP = \frac{VP}{P} = """+r"{:.2f}\ \% $$".format(Se2.value*ma/P2.value)
    VPP2[-1].value = r"""On peut calculer la valeur prédictive négative VPN car on connait <b>le nombre de vrais négatifs VN</b> et le nombre de <b>tests négatifs N</b> dans l'échantillon:
$$VPN = \frac{VN}{N} = """+r"{:.2f}\ \% $$".format((100.*N2.value-(100.-Se2.value)*ma)/N2.value)
    DE2[0].value = r"""On peut calculer le pourcentage de diagnostiques corrects car on connait <b>le nombre de vrais positifs VP</b> et <b>le nombre de vrais négatifs VN</b> dans l'échantillon:
$$Dc = \frac{VP + VN}{Nombre\ total\ de\ tests} = """+r"{:.2f}\ \% $$".format((Se2.value*ma+100.*N2.value-(100.-Se2.value)*ma)/(P2.value+N2.value))
    DE2[-1].value = r"""On peut calculer le pourcentage de diagnostiques erronés car on connait <b>le nombre de faux positifs FP</b> et <b>le nombre de faux négatifs FN</b> dans l'échantillon:
$$De = \frac{FP + FN}{Nombre\ total\ de\ tests} = """+r"{:.2f}\ \% $$".format(((100.-Se2.value)*ma+100.*P2.value-Se2.value*ma)/(P2.value+N2.value))
    
Se2.observe(change2, names='value')
Sp2.observe(change2, names='value')
P2.observe(change3, names='value')
N2.observe(change3, names='value')

A21 = Accordion()
children21 = [VBox([HTMLMath(r"""On sait (revoir le 1.4. si besoin) que le nombre de tests positifs P es tlié au nombre de malades dans l'échantillon, et à la sélectivité et à la spécificité du test par:
$P = Se \times M + (1 - Sp) \times S$
</br></br>
De même pour les tests négatifs on a:
$N = Sp \times S + (1 - Se) \times M$
</br></br>
On a ainsi deux équations avec deux inconnues M et S (tout le reste est connu) que l'on peut donc déterminer."""),
                   HTML("",layout=l),
                   HBox(tableau4,layout=Layout(width='600px',height='150px')),
                   HTML("",layout=l),
                   HTML("Si l'échantillon testé est représentatif de la population, on a accès à la prévalence de la maladie:"),
                   HBox(Pr2,layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='center',
                    align_content = 'center'))],layout=Layout(width='95%'))]

children21.append(VBox([HTMLMath(r"""On peut calculer le nombre de VP et FN car on connait <b>la sensibilité Se</b> du test et <b>le nombre de personnes malades M</b> dans l'échantillon:
$Se = \frac{VP}{VP + FN} = \frac{VP}{M}$ d'où $VP = Se \times M$ et $FN = M - VP$"""),
                        HTML("",layout=l),
                        HTMLMath(r"""De même on peut calculer le nombre de VN et FP car on connait <b>la spécificité Sp</b> du test et <b>le nombre de personnes bien portantes S</b> dans l'échantillon:
$Sp = \frac{VN}{VN + FP} = \frac{VN}{S}$ d'où $VN = Sp \times S$ et $FP = S - VN$"""),
                        HTML("",layout=l),
                        HBox(tableau5,layout=Layout(width='600px',height='150px'))],layout=Layout(width='95%')))

children21.append(VBox(VPP2,layout=Layout(width='95%')))
children21.append(VBox(DE2,layout=Layout(width='95%')))

#children21.append(VBox([HBox(M), HTML("",layout=l),HBox(S)]))

A21.children = children21
A21.set_title(0,"3.1. Détermination du nombre de personnes malades et prévalence")
A21.set_title(1,"3.2. Nombre de VP, VN, FP et FN")
A21.set_title(2,"3.3. Valeur prédictive positive et Valeur prédictive négative")
A21.set_title(3,"3.4. Diagnostique correct/erroné")
A21.selected_index=None

children1.append(VBox([HTML("Modifiez les valeurs en cliquant sur les sliders, vous pouvez utiliser les flèches du clavier afin d'affiner les valeurs"),
                       HBox([VBox([VP,FP,VN,FN],layout=Layout(width='50%')),HBox(tableau,layout=Layout(width='400px'))],layout=Layout(width='95%', height='150px')),A11]))

children1.append(VBox([HTML("""On fait passer le test à un échantillon de personnes (ici 1248 personnes) dont on sait qu'elles sont malades et on mesure le nombre de résultats positifs et négatifs.
                            On fait de même sur un échantillon de personnes (3927) dont on sait qu'elles sont bien portantes.
                            Ces résultats permettent de connaitre les caractéristiques du test comme <b>la sensibilité et la spécificité</b> du test c'est à dire la probabilité d'avoir un test
                            positif sachant que l'on est malade et la probabilité d'avoir un test négatif sachant que l'on est bien portant. L'inférence bayésienne nous permettra ensuite l'utilisation du test afin d'obtenir sur un échantillon de population testée
                            la probabilité d'être malade sachant que le test est positif et la probabilité d'être bien portant sahant que le test est négatif."""),
                       Se2,HTML("",layout=l),Sp2,HBox(tableau2,layout=Layout(width='600px',height='120px'))],layout=Layout(width='95%', height='400px')))

children1.append(VBox([HTML("""On fait maintenant passer le test dont <b>on connait la sélectivité Se et la spécificité Sp</b> à un échantillon de population. On mesure le nombre de résultats positifs et le nombre de résultats négatifs.  
</br>L'inférence bayésienne va alors nous permettre d'accéder à beaucoup d'informations bien que le test ne soit pas parfait."""),Se2_res,Sp2_res,HTML("",layout=l),P2,HTML("",layout=l),N2,HTML("",layout=l),HBox(tableau3,layout=Layout(width='600px',height='150px')),A21],layout=Layout(width='95%')))







A1.children = children1
A1.set_title(0,"1. Comprendre les notions")
A1.set_title(1,"2. Mise au point d'un test")
A1.set_title(2,"3. Utilisation d'un test")
display(A1)

